# Fine Tune

**Reference:** https://neptune.ai/blog/hugging-face-pre-trained-models-find-the-best

In [1]:
# !pip install -r requirements.txt -i https://pypi.douban.com/simple

In [2]:
# !git clone https://github.com/huggingface/evaluate.git

In [10]:
import pandas as pd
from datasets import Dataset

import warnings
warnings.filterwarnings("ignore")

from Translate import Translate

pd.set_option('display.max_colwidth', None)

data_proc = "../data/trans/PROC_2023.csv"

model_mbart = "facebook/mbart-large-50-many-to-one-mmt"
model_name = f"mbart-finetuned-cn-to-en-auto"
model_path = f"../models/{model_name}"

finetuned_model_path = model_path

## Load Data

In [4]:
df = pd.read_csv(data_proc)

In [5]:
df.head()

,Chinese,English
0,开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快,"In the case of turning on the air conditioner, the electric range drops too fast, especially when the weather is cold in winter, if you don't turn on the air conditioner, the weather freezes, the battery life will fall faster."
1,车机流畅度差，容易卡死机，车机系统，启动载入很慢，换挡杆前的车机，使用任何功能都有概率死机，发生过3-4次,"The smoothness of the IHU is poor, easy to jam, the car machine system, the start loading is very slow, the car machine before the gear lever, using any function has a probability of crashing, which has occurred 3-4 times."
2,整车的悬架系统，在过减速带时，速度在20码以下，但是车身的抖动还是很厉害，舒适性为第一的，美系车相比，差距还是比较大的,"The suspension system of the whole car, when crossing the speed bump, the speed is below 20km/h, but the shaking of the body is still very strong, the comfort is the first, compared with the American car, the gap is still relatively large."
3,大众车的通病，车子的隔音效果不太理想，车速在90码以上，车内的胎噪声就很明显了，必须把音量调大，才能缓解一点（是原厂轮胎，车窗关闭）,"The common problem of Volkswagen, the sound insulation of the car is not ideal, the speed is above 90km/h, the tire noise in the car is obvious, the volume must be turned up, in order to alleviate a little (is the original tires, the windows are closed."
4,车辆外观很不错，但是车标在晚上不能发亮，要是可以发亮的话会更拉风一点,"The appearance of the vehicle is very good, but the logo cannot be shiny at night, if it can be bright, it will be more stunning."


### Batch Processing

In [6]:
trans = Translate(model_mbart)
text = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"
print(trans.translator(text))

If the air conditioner is on, the flight resumes too quickly, especially in winter when the weather is cold. If the air conditioner is not on, the flight resumes faster as soon as the weather is cold


## Fine-tune the Model

In [7]:
trans.finetune(df, finetuned_model_path=finetuned_model_path, compute_metrics=False)

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.755436


{'eval_loss': 0.6907669901847839, 'eval_runtime': 4.0246, 'eval_samples_per_second': 5.715, 'eval_steps_per_second': 1.491, 'epoch': 1.0}


## Test
### Single Case

In [8]:
trans = Translate(finetuned_model_path)

text = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"
print(trans.translator(text))

In the case of turning on the air conditioner, the electric range drops too fast, especially when the weather is cold in winter, not turning on the air conditioner is not possible, the weather freezes, and the electric range drops faster.


> **ORIGINAL MBART:** If the air conditioner is on, the flight resumes too quickly, especially in winter when the weather is cold. If the air conditioner is not on, the flight resumes faster as soon as the weather is cold

### Batch Processing

In [9]:
df = trans.translator_batch(df.head(10), col_tgt="Translation")
df.head()

100%|██████████| 10/10 [00:37<00:00,  3.73s/it]


,Chinese,English,Translation
0,开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快,"In the case of turning on the air conditioner, the electric range drops too fast, especially when the weather is cold in winter, if you don't turn on the air conditioner, the weather freezes, the battery life will fall faster.","In the case of turning on the air conditioner, the electric range drops too fast, especially when the weather is cold in winter, not turning on the air conditioner is not possible, the weather freezes, and the electric range drops faster."
1,车机流畅度差，容易卡死机，车机系统，启动载入很慢，换挡杆前的车机，使用任何功能都有概率死机，发生过3-4次,"The smoothness of the IHU is poor, easy to jam, the car machine system, the start loading is very slow, the car machine before the gear lever, using any function has a probability of crashing, which has occurred 3-4 times.","The smoothness of the IHU is poor, it is easy to jam the IHU, the IHU system, the start loading is slow, the IHU before the gear lever, the use of any function has a probability of crashing, which has occurred 3-4 times."
2,整车的悬架系统，在过减速带时，速度在20码以下，但是车身的抖动还是很厉害，舒适性为第一的，美系车相比，差距还是比较大的,"The suspension system of the whole car, when crossing the speed bump, the speed is below 20km/h, but the shaking of the body is still very strong, the comfort is the first, compared with the American car, the gap is still relatively large.","The suspension system of the whole car, when crossing the speed bump, the speed is less than 20km/h, but the body shaking is still very bad, comfort is the first, compared to the American car, the gap is still relatively large."
3,大众车的通病，车子的隔音效果不太理想，车速在90码以上，车内的胎噪声就很明显了，必须把音量调大，才能缓解一点（是原厂轮胎，车窗关闭）,"The common problem of Volkswagen, the sound insulation of the car is not ideal, the speed is above 90km/h, the tire noise in the car is obvious, the volume must be turned up, in order to alleviate a little (is the original tires, the windows are closed.","The common problem of mass-produced cars, the sound insulation effect of the car is not ideal, the speed is above 90km/h, the tire noise in the car is very obvious, it is necessary to turn up the volume, in order to alleviate a little (is the original tire, the window is closed."
4,车辆外观很不错，但是车标在晚上不能发亮，要是可以发亮的话会更拉风一点,"The appearance of the vehicle is very good, but the logo cannot be shiny at night, if it can be bright, it will be more stunning.","The appearance of the vehicle is very good, but the license plate cannot shine at night, if it can shine, it will be more attractive."
